In [2]:
import serial

def Main() -> None:
  ser = serial.Serial('COM3', 9600)
  try:
    while True:
      ReadData(ser)
  except KeyboardInterrupt:
    ser.close()
    return

Main()

SSLError: HTTPSConnectionPool(host='localhost', port=7210): Max retries exceeded with url: /api/PlantsHist (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1129)')))

In [1]:
from IPython.display import clear_output
from datetime import datetime
import requests

def ReadData(ser):
  data = []                       # empty list to store the data
  for i in range(4):
    b = ser.readline()            # read a byte string
    string_n = b.decode()         # decode byte string into Unicode  
    string = string_n.rstrip()    # remove \n and \r
    data.append(string)           # add to the end of data list
  clear_output()
  response = requests.post("https://localhost:7210/api/PlantsHist", json={ "plantID": 1, "sunlight": int(data[0]) > 350, "temperature": data[2], "airHumidity": data[1], "soilHumidity": data[3]})
  print(response.status_code)
  print(f"naświetlenie: {data[0]}, wilgotność powietrza: {data[1]}, temperatura: {data[2]}, wilgotność gleby: {data[3]}, czas: {datetime.now().__str__()}")

poniżej stare funkcje do dodawania prosto do bazy danych

In [1]:
import sqlite3
conn=sqlite3.connect('arduinoSensors.db')
conn.row_factory=sqlite3.Row
c=conn.cursor()

In [12]:
c.execute("DROP TABLE IF EXISTS sensorInfo;")
c.execute("""
    CREATE TABLE IF NOT EXISTS sensorInfo (
        id INTEGER PRIMARY KEY ASC,
        photoresistor varchar(10) NOT NULL,
        thermometer varchar(10) NOT NULL,
        moisturemeter varchar(10) DEFAULT ''
    )""")

In [10]:
from datetime import datetime

def addToDB(photo, thermo, moisture):
  c.execute('INSERT INTO sensorInfo VALUES(NULL, ?, ?, ?);', (photo, thermo, moisture))

In [11]:
def showTable():
    c.execute("SELECT sensorInfo.id,photoresistor,thermometer,moisturemeter FROM sensorInfo")
    sensorInfo = c.fetchall() #pobieramy wszystkie pasujące rekordy
    for info in sensorInfo:
        print(info['id'], info['photoresistor'], info['thermometer'], info['moisturemeter'])

In [4]:
showTable()